To Dos: 1. Data Generation 2. Plotte Density 3. Netzwerk definieren (probiere auch zusätzlich F in 0,1 zu constrainen) 4. Ableitung des Netzwerks bestimmen und plotten als Density Estimate und das mit dem echten vergleichen

# Load Packages


In [ ]:
import numpy as np
from scipy import stats          # sampling

import matplotlib.pyplot as plt  # visualization

import torch                     # constructing, learning, using NNs
import torch.nn as nn
import torch.optim as optim

# Data Generation


In [ ]:
# construct 2-component Gaussian mixture distribution
comp1 = stats.Normal( mu=2, sigma=0.25)
comp2 = stats.Normal( mu=6, sigma=0.5)
mix = stats.Mixture( [comp1, comp2], weights=[0.2, 0.8])

#n = 100
#x = mix.sample(n)

# Visualization of the analytic PDF


In [ ]:
#| echo: false

x = np.linspace(0, 10, 1000)
plt.clf()
plt.plot(x, mix.pdf(x))
plt.title("Analytic PDF")
plt.show()

In [ ]:
from scipy import stats

n = 100

# construct 2-component Gaussian mixture distribution
comp1 = stats.Normal(mu=-5, sigma=1)
comp2 = stats.Normal(mu=5, sigma=1)
mixture = stats.Mixture([comp1,comp2],weights=[0.4, 0.6])

# sample from mixture distribution, sort and make torch tensor 
x = mixture.sample(n)
x = torch.from_numpy(np.sort(x)).float().unsqueeze(1)

In [ ]:
u = np.random.uniform(0, 1, n)
u = torch.from_numpy(np.sort(u)).float().unsqueeze(1)


import torch
import torch.nn as nn
import torch.optim as optim


class Network(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Network, self).__init__()
    self.net = nn.Sequential(
      nn.Linear(input_size, hidden_size),
      nn.Sigmoid(),
      nn.Linear(hidden_size, output_size)
      )
  
  def forward(self, x):
    return self.net(x)


class CustomLoss(nn.Module):
  def __init__(self):
    super(CustomLoss, self).__init__()
  
  def forward(self, predictions, targets):
    loss_squared = torch.mean((predictions - targets)**2)
    return loss_squared


class CustomLoss2(nn.Module):
  def __init__(self):
    super(CustomLoss2, self).__init__()
  
  def forward(self, y_pred, y_true, lambda_mon, mon_d, mon_u):
    loss_prediction = torch.mean((y_pred - y_true)**2)
    
    diff = mon_d - mon_u
    penalty = torch.sigmoid(10000 * diff) * (diff**2)
    loss_monotonicity = torch.mean(penalty)
    
    #loss_monotonicity = torch.mean(torch.heaviside(mon_d - mon_u, torch.tensor(0.0))*(mon_d - mon_u)**2)
    loss_total = loss_prediction + lambda_mon * loss_monotonicity
    return loss_total

model = Network(input_size=1, hidden_size=30, output_size=1)
#criterion = CustomLoss()
criterion = CustomLoss2()
optimizer = optim.Adam(model.parameters(), lr=0.001)


lambda_mon = 1000000
n_mon_points = 1000
mon_points = torch.linspace(x[0,0], x[-1,0], n_mon_points)[:, None]
delta = 0.1*(max(x)-min(x))/n_mon_points

num_epochs = 100
steps_per_epoch = 200


for epoch in range(num_epochs):
  for step in range(steps_per_epoch):
    preds = model(x)
    u = np.random.uniform(0, 1, n)
    u = torch.from_numpy(np.sort(u)).float().unsqueeze(1)
    
    mon_d = model(mon_points)
    mon_u = model(mon_points + delta)
    
    #loss = criterion(preds, u)
    
    loss = criterion(preds, u, lambda_mon, mon_d, mon_u)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


import matplotlib.pyplot as plt
xx = torch.linspace(-10, 10, 500).unsqueeze(1)
with torch.no_grad():
    yy = model(xx)

plt.figure(figsize=(10, 5))
plt.plot(x.numpy(), u.numpy(), label="Target (sorted uniform)", color='green')
plt.plot(xx.numpy(), yy.numpy(), label="Learned CDF", color='blue')
plt.plot(xx.numpy(), mixture.cdf(xx), label="Learned CDF", color='red')
plt.title("NN fitting empirical CDF")
plt.legend()
plt.show()

In [ ]:
from scipy import stats
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

n = 500

comp1 = stats.norm(loc=-5, scale=1)  # Fix: stats.Normal → stats.norm with loc and scale
comp2 = stats.norm(loc=5, scale=1)
x = np.concatenate([
    comp1.rvs(size=int(0.4 * n)),
    comp2.rvs(size=int(0.6 * n))
])
np.random.shuffle(x)
x = torch.from_numpy(np.sort(x)).float().unsqueeze(1)

u = np.random.uniform(0, 1, n)
u = torch.from_numpy(np.sort(u)).float().unsqueeze(1)

class Network(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Network, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Sigmoid(),
            nn.Linear(hidden_size, output_size)
        )
    
    def forward(self, x):
        return self.net(x)

class CustomLoss2(nn.Module):
    def __init__(self):
        super(CustomLoss2, self).__init__()
    
    def forward(self, y_pred, y_true, lambda_mon, mon_d, mon_u):
        loss_prediction = torch.mean((y_pred - y_true)**2)
        diff = mon_d - mon_u
        penalty = torch.clamp(diff, min=0.0)**2  # Use clamp to penalize downward slopes
        loss_monotonicity = torch.mean(penalty)
        loss_total = loss_prediction + lambda_mon * loss_monotonicity
        return loss_total

model = Network(input_size=1, hidden_size=30, output_size=1)
criterion = CustomLoss2()
optimizer = optim.Adam(model.parameters(), lr=0.001)

lambda_mon = 1000000  # You can tune this
n_mon_points = 1000
mon_points = torch.linspace(x[0,0], x[-1,0], n_mon_points)[:, None]
delta = 0.1*(max(x)-min(x))/n_mon_points

num_epochs = 100
steps_per_epoch = 200

for epoch in range(num_epochs):
    for step in range(steps_per_epoch):
        preds = model(x)
        u = np.random.uniform(0, 1, n)
        u = torch.from_numpy(np.sort(u)).float().unsqueeze(1)
        
        mon_d = model(mon_points)
        mon_u = model(mon_points + delta)
        
        loss = criterion(preds, u, lambda_mon, mon_d, mon_u)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

xx = torch.linspace(-10, 10, 500).unsqueeze(1)
with torch.no_grad():
    yy = model(xx)

plt.figure(figsize=(10, 5))
plt.plot(x.numpy(), u.numpy(), label="Target (sorted uniform)", color='green')
plt.plot(xx.numpy(), yy.numpy(), label="Learned CDF", color='blue')
plt.plot(xx.numpy(), 0.4 * comp1.cdf(xx.numpy()) + 0.6 * comp2.cdf(xx.numpy()), label="True CDF (Mixture)", color='red')
plt.title("NN fitting empirical CDF")
plt.legend()
plt.show()

In [ ]:
#### DATA GENERATION ###

from scipy import stats

n = 100

comp1 = stats.Normal(mu=-2, sigma=1)
comp2 = stats.Normal(mu=2, sigma=1)
mixture = stats.Mixture([comp1,comp2],weights=[0.4, 0.6])

x = mixture.sample(n)

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
X1 = stats.Normal(mu=-2, sigma=1)
X2 = stats.Normal(mu=2, sigma=1)
mixture = stats.Mixture([X1,X2],weights=[0.4, 0.6])
x = np.linspace(-10, 10, 300)
plt.clf()
plt.plot(x, mixture.pdf(x))
plt.title('XXXX of normal distribution mixture')
plt.show()

plt.clf()
plt.hist(mixture.sample(1000), bins=20, density=True, alpha=0.7, color='blue')
plt.show()


# To Dos:
# 1. define mixture
# 2. sample from mixture